In [1]:
""" Example script for predicting columns of tables, demonstrating simple use-case """

from autogluon.tabular import TabularDataset, TabularPredictor


# Training time:
train_data = TabularDataset('../../modified/monthly-airline-passengers-train.csv')  # can be local CSV file as well, returns Pandas DataFrame
train_data = train_data.head(500)  # subsample for faster demo
print(train_data.head())
label = 'class'  # specifies which column do we want to predict
save_path = 'monthly_airline_passengers_ms/'  # where to save trained models



         0  class
0  1954-07    302
1  1960-03    419
2  1958-10    359
3  1951-05    172
4  1957-09    404


In [2]:
predictor = TabularPredictor(label=label, path=save_path).fit(train_data, time_limit=3600)
# NOTE: Default settings above are intended to ensure reasonable runtime at the cost of accuracy. To maximize predictive accuracy, do this instead:
# predictor = TabularPredictor(label=label, eval_metric=YOUR_METRIC_NAME, path=save_path).fit(train_data, presets='best_quality')
results = predictor.fit_summary()


Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "monthly_airline_passengers_ms/"
AutoGluon Version:  0.2.0
Train Data Rows:    115
Train Data Columns: 1
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (622, 104, 280.27826, 121.7046)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    510056.61 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFe

[1000]	train_set's rmse: 47.6893	valid_set's rmse: 44.1087
[2000]	train_set's rmse: 47.3477	valid_set's rmse: 43.9162


	-43.8825	 = Validation root_mean_squared_error score
	2.93s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 3595.53s of the 3595.53s of remaining time.
	-52.4017	 = Validation root_mean_squared_error score
	0.84s	 = Training runtime
	0.12s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 3594.56s of the 3594.56s of remaining time.
	-53.715	 = Validation root_mean_squared_error score
	0.18s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 3594.37s of the 3594.37s of remaining time.
	-52.1288	 = Validation root_mean_squared_error score
	0.94s	 = Training runtime
	0.12s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 3593.29s of the 3593.28s of remaining time.
	-302.2315	 = Validation root_mean_squared_error score
	3.97s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost ... Training model fo

*** Summary of fit() ***
Estimated performance of each model:
                 model   score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2  -40.729976       0.247097  11.807750                0.000610           0.423252            2       True         10
1             LightGBM  -43.882464       0.012847   2.925854                0.012847           2.925854            1       True          2
2              XGBoost  -51.353526       0.002909   0.352035                0.002909           0.352035            1       True          7
3        ExtraTreesMSE  -52.128770       0.119126   0.943112                0.119126           0.943112            1       True          5
4      RandomForestMSE  -52.401664       0.116745   0.836100                0.116745           0.836100            1       True          3
5             CatBoost  -53.714984       0.003253   0.182872                0.003253           0.182872 

In [3]:
# Inference time:
test_data = TabularDataset('../../modified/monthly-airline-passengers-test.csv')  # another Pandas DataFrame
y_test = test_data[label]
test_data = test_data.drop(labels=[label], axis=1)  # delete labels from test data since we wouldn't have them in practice
print(test_data.head())



Loaded data from: ../../modified/monthly-airline-passengers-test.csv | Columns = 2 / 2 | Rows = 29 -> 29


         0
0  1954-02
1  1956-05
2  1956-11
3  1958-05
4  1950-05


In [4]:
predictor = TabularPredictor.load(save_path)  # Unnecessary, we reload predictor just to demonstrate how to load previously-trained predictor from file
y_pred = predictor.predict(test_data)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


Evaluation: root_mean_squared_error on test data: -39.39104401378377
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -39.39104401378377,
    "mean_squared_error": -1551.65434849585,
    "mean_absolute_error": -28.988385430697736,
    "r2": 0.8782109828689734,
    "pearsonr": 0.9383370926842507,
    "median_absolute_error": -25.0328369140625
}


In [7]:
print('Accuracy = {:.2f}'.format(perf["root_mean_squared_error"]))

Accuracy = -39.39


In [8]:
train_data.tail()

,0,class
110,1950-12,140
111,1953-07,264
112,1949-07,148
113,1959-05,420
114,1958-01,340


In [9]:
predictor = TabularPredictor(label=label, path=save_path).fit(train_data, time_limit=3600)
# NOTE: Default settings above are intended to ensure reasonable runtime at the cost of accuracy. To maximize predictive accuracy, do this instead:
# predictor = TabularPredictor(label=label, eval_metric=YOUR_METRIC_NAME, path=save_path).fit(train_data, presets='best_quality')
results = predictor.fit_summary()
import autogluon.core as ag

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'layers': ag.space.Categorical([100], [1000], [200, 100], [300, 200, 100]),  # each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 2*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using Bayesian optimization routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

path = 'monthly_airline_passengers_additional_hypers_ms/'  # where to save trained models

predictor = TabularPredictor(label=label, path=path).fit(
    train_data, time_limit=3600,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)


Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "monthly_airline_passengers_ms/"
AutoGluon Version:  0.2.0
Train Data Rows:    115
Train Data Columns: 1
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (622, 104, 280.27826, 121.7046)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    507970.0 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFea

[1000]	train_set's rmse: 47.6893	valid_set's rmse: 44.1087
[2000]	train_set's rmse: 47.3477	valid_set's rmse: 43.9162


	-43.8825	 = Validation root_mean_squared_error score
	6.3s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 3591.01s of the 3591.0s of remaining time.
	-52.4017	 = Validation root_mean_squared_error score
	0.86s	 = Training runtime
	0.12s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 3590.01s of the 3590.01s of remaining time.
	-53.715	 = Validation root_mean_squared_error score
	0.1s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 3589.9s of the 3589.9s of remaining time.
	-52.1288	 = Validation root_mean_squared_error score
	0.85s	 = Training runtime
	0.12s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 3588.91s of the 3588.91s of remaining time.
	-302.624	 = Validation root_mean_squared_error score
	0.84s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost ... Training model for up t

*** Summary of fit() ***
Estimated performance of each model:
                 model   score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2  -41.156763       0.123720  7.591243                0.000678           0.429753            2       True         10
1             LightGBM  -43.882464       0.006022  6.301794                0.006022           6.301794            1       True          2
2              XGBoost  -51.353526       0.003671  1.203275                0.003671           1.203275            1       True          7
3        ExtraTreesMSE  -52.128770       0.118657  0.850056                0.118657           0.850056            1       True          5
4      RandomForestMSE  -52.401664       0.117020  0.859696                0.117020           0.859696            1       True          3
5             CatBoost  -53.714984       0.002997  0.103866                0.002997           0.103866        

		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('object', ['datetime_as_object']) : 1 | ['0']
	Types of features in processed data (raw dtype, special dtypes):
		('int', ['datetime_as_int']) : 1 | ['0']
	0.0s = Fit runtime
	1 features in original data used to generate 1 features in processed data.
	Train Data (Processed) Memory Usage: 0.0 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.06s ...
AutoGluon will gauge predictive performance using evaluation metric: 'root_mean_squared_error'
	To change this, specify the eval_metric argument of fit()
Automatically generating train/validation split with holdout_frac=0.2, Train Rows: 92, Val Rows: 23
Hyperparameter tuning model: LightGBM ...
100%|██████████| 5/5 [00:03<00:00,  1.31it/s]
Fitted model: LightGBM/T0 ...
	-44.7611	 = Validation root_mean_squared_error score
	1.51s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T1

In [10]:
predictor = TabularPredictor.load(path)  # Unnecessary, we reload predictor just to demonstrate how to load previously-trained predictor from file
y_pred = predictor.predict(test_data)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -41.46283804397411
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -41.46283804397411,
    "mean_squared_error": -1719.1669386608269,
    "mean_absolute_error": -30.958954909752155,
    "r2": 0.8650629555824572,
    "pearsonr": 0.9354652075779957,
    "median_absolute_error": -20.561065673828125
}


In [ ]:
# Evaluations on test data:
{
    "root_mean_squared_error": -39.39104401378377,
    "mean_squared_error": -1551.65434849585,
    "mean_absolute_error": -28.988385430697736,
    "r2": 0.8782109828689734,
    "pearsonr": 0.9383370926842507,
    "median_absolute_error": -25.0328369140625
}